## Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
from bs4 import BeautifulSoup

In [4]:
import requests

## Loading the page

In [5]:
link="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [6]:
raw_page=requests.get(link)

In [7]:
page=raw_page.text

In [9]:
#print(page)

## Website scraping using BeautifulSoup

In [8]:
soup = BeautifulSoup(page,"lxml")

In [10]:
#print(soup.prettify())

In [9]:
body = soup.find("tbody")

In [61]:
#print(body.prettify())

In [10]:
rows=body("tr")

allRowItems=BeautifulSoup("<items></items>","lxml")
for row in rows:
    rowItems = row("td")
    for rowItem in rowItems:
        allRowItems.items.append(rowItem)

print(allRowItems.prettify())

<html>
 <body>
  <items>
   <td style="width:11%; vertical-align:top; color:#ccc;">
    <p>
     <b>
      M1A
     </b>
     <br/>
     <span style="font-size:80%;">
      <i>
       Not assigned
      </i>
     </span>
    </p>
   </td>
   <td style="width:11%; vertical-align:top; color:#ccc;">
    <p>
     <b>
      M2A
     </b>
     <br/>
     <span style="font-size:80%;">
      <i>
       Not assigned
      </i>
     </span>
    </p>
   </td>
   <td style="width:11%; vertical-align:top;">
    <p>
     <b>
      M3A
     </b>
     <br/>
     <span style="font-size:80%;">
      <a href="/wiki/North_York" title="North York">
       North York
      </a>
      <br/>
      (
      <a href="/wiki/Parkwoods" title="Parkwoods">
       Parkwoods
      </a>
      )
     </span>
    </p>
   </td>
   <td style="width:11%; vertical-align:top;">
    <p>
     <b>
      M4A
     </b>
     <br/>
     <span style="font-size:80%;">
      <a href="/wiki/North_York" title="North York">
       North Y

In [11]:
tds=allRowItems.items("td")
tds
records=[]
for td in tds:
   
    zip_code =td.p.b.text  
    
    texts=td.span.find_all("a")
    
    if len(texts)==0:
        borough="Not assigned"
        neighborhood = "Not assigned"
    elif len(texts)==1:
        borough=texts[0].text
        neighborhood = texts[0].text
    else:
        borough=texts[0].text
        neighborhood=""
        for i in range(len(texts)-1):
            
            neighborhood_part = texts[i+1].text
            if i<(len(texts)-2):
                
                neighborhood+=neighborhood_part+", "
            else:
                neighborhood+=neighborhood_part
            
            
    
    records.append([zip_code,borough,neighborhood])
    print(zip_code)
    print(borough)
    print(neighborhood)
    print()

M1A
Not assigned
Not assigned

M2A
Not assigned
Not assigned

M3A
North York
Parkwoods

M4A
North York
Victoria Village

M5A
Downtown Toronto
Regent Park, Harbourfront

M6A
North York
Lawrence Manor, Lawrence Heights

M7A
Queen's Park
Queen's Park

M8A
Not assigned
Not assigned

M9A
Etobicoke
Islington Avenue

M1B
Scarborough
Malvern, Rouge

M2B
Not assigned
Not assigned

M3B
North York
Don Mills

M4B
East York
Parkview Hill, Woodbine Gardens

M5B
Downtown Toronto
Garden District, Ryerson

M6B
North York
North York

M7B
Not assigned
Not assigned

M8B
Not assigned
Not assigned

M9B
Etobicoke
West Deane Park, Princess Gardens, Martin Grove, Islington

M1C
Scarborough
Rouge Hill, Port Union, Highland Creek

M2C
Not assigned
Not assigned

M3C
North York
Don Mills, Flemingdon Park

M4C
East York
Woodbine Heights

M5C
Downtown Toronto
St. James Town

M6C
York
Humewood-Cedarvale

M7C
Not assigned
Not assigned

M8C
Not assigned
Not assigned

M9C
Etobicoke
Markland Wood

M1E
Scarborough
Guildwo

In [11]:
len(records)

180

## Creating DF

In [12]:
cols=["PostalCode","Borough","Neighborhood"]

In [13]:
toronto = pd.DataFrame(records,columns=cols)

In [14]:
toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [15]:
(toronto["Borough"]=="Not assigned").value_counts()

False    101
True      79
Name: Borough, dtype: int64

In [16]:
toronto.drop(toronto[toronto["Borough"]=="Not assigned"].index, inplace=True)

In [17]:
toronto.reset_index(inplace=True,drop=True)

In [18]:
toronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park


## Shape of the data frame

In [19]:
toronto.shape

(101, 3)

## Getting Lat and Lng

In [20]:
# The code was removed by Watson Studio for sharing.

In [24]:
for i in range(len(toronto)):
    url='http://dev.virtualearth.net/REST/v1/Locations?postalCode={}&key={}'.format(toronto["PostalCode"][i],API_key)
    response=requests.get(url).json()
    g_data = response["resourceSets"][0]["resources"][0]["geocodePoints"][0]["coordinates"]
    toronto["Latitude"][i]=g_data[0]
    toronto["Longitude"][i]=g_data[1]
    i=i+1

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [124]:
#toronto["PostalCode"][0]

In [22]:
toronto["Latitude"]=np.nan

In [23]:
toronto["Longitude"]=np.nan

In [125]:
#toronto.head()

In [126]:
#toronto[toronto["PostalCode"]=="M4B"]

In [120]:
#response=requests.get(url1).json()
#response

In [121]:
#response.keys()

In [122]:
#response["resourceSets"][0]["resources"][0]["geocodePoints"][0]["coordinates"]

In [25]:
toronto.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.751255,-79.329895
1,M4A,North York,Victoria Village,43.729958,-79.314201
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655220,-79.361969
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722801,-79.450691
4,M7A,Queen's Park,Queen's Park,43.664486,-79.393021
5,M9A,Etobicoke,Islington Avenue,43.662743,-79.528427
6,M1B,Scarborough,"Malvern, Rouge",43.810154,-79.194603
7,M3B,North York,Don Mills,43.749134,-79.362007
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.707577,-79.310913
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657467,-79.377708


## Exploring the neighborhoods of Toronto

In [26]:
print("Toronto has {} boroughs and {} neighborhoods.".format(len(toronto["Borough"].unique()),len(toronto)))

Toronto has 25 boroughs and 101 neighborhoods.


### Importing libraries 2

In [27]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.16.0                     py_0    conda-forge


In [35]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


### Getting coordinates of Toronto, ON

In [28]:
address = 'Toronto,ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:138: UserWarning: Using Nominatim with the default "geopy/1.16.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Map of Toronto with neighborhoods

In [34]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Downtown Toronto

In [29]:
downtown_t=toronto[toronto["Borough"]=="Downtown Toronto"]

In [30]:
downtown_t.reset_index(inplace=True,drop=True)

In [31]:
downtown_t

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655220,-79.361969
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657467,-79.377708
2,M5C,Downtown Toronto,St. James Town,43.651211,-79.375481
3,M5E,Downtown Toronto,Downtown Toronto,43.643330,-79.372223
4,M5G,Downtown Toronto,Bay Street,43.656078,-79.384918
5,M6G,Downtown Toronto,Downtown Toronto,43.668850,-79.420380
6,M5H,Downtown Toronto,"Richmond, King",43.650410,-79.382881
7,M5J,Downtown Toronto,"Harbourfront, Union Station, Toronto Islands",43.617500,-79.372826
8,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.648106,-79.382439
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648403,-79.379227


In [32]:
print("There are {} neighborhoods in Downtown Toronto.".format(len(downtown_t["Neighborhood"].unique())))

There are 16 neighborhoods in Downtown Toronto.


In [33]:
address1 = 'Downtown Toronto, Toronto'

geolocator1 = Nominatim()
location1 = geolocator.geocode(address)
latitude1 = location.latitude
longitude1 = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:138: UserWarning: Using Nominatim with the default "geopy/1.16.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Downtown Toronto are 43.653963, -79.387207.


### Map of Downtown Toronto

In [37]:
map_dt = folium.Map(location=[latitude1, longitude1], zoom_start=13)

for lat, lng, borough, neighborhood in zip(downtown_t['Latitude'], downtown_t['Longitude'], downtown_t['Borough'], downtown_t['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='green',
        fill_opacity=0.6,
        parse_html=False).add_to(map_dt)  
    
map_dt

## Neighborhood venues in Downtown Toronto

### API Credentials

In [38]:
# The code was removed by Watson Studio for sharing.

### Explore top 100 venues in Downtown Toronto

In [39]:
limit=100

def DT_50_Venues(names, latitudes, longitudes, radius=500):
    
    venueslist=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venueslist.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    DT_venues = pd.DataFrame([item for venuelist in venueslist for item in venuelist])
    DT_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(DT_venues)

In [40]:
dt_toronto_venues=DT_50_Venues(downtown_t["Neighborhood"],downtown_t["Latitude"],downtown_t["Longitude"])

In [41]:
dt_toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65522,-79.361969,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65522,-79.361969,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65522,-79.361969,Body Blitz Spa East,43.654735,-79.359874,Spa
3,"Regent Park, Harbourfront",43.65522,-79.361969,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
4,"Regent Park, Harbourfront",43.65522,-79.361969,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [61]:
print("Total number of venues: {}".format(len(dt_toronto_venues)))
print("Total number of unique venues: {}".format(len(dt_toronto_venues["Venue Category"].unique())))

Total number of venues: 1220
Total number of unique venues: 181


### Venues by neighborhood

In [42]:
dt_toronto_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bay Street,100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport",66,66,66,66,66,66
Church and Wellesley,82,82,82,82,82,82
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Downtown Toronto,127,127,127,127,127,127
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront, Union Station, Toronto Islands",10,10,10,10,10,10
"Kensington Market, Chinatown, Grange Park",100,100,100,100,100,100


### Top 10 venues in Downtown Toronto

In [107]:
dt_toronto_venues["Venue Category"].value_counts().head(10)

Coffee Shop            104
Café                    65
Restaurant              43
Hotel                   42
Italian Restaurant      36
Japanese Restaurant     30
Bar                     30
American Restaurant     28
Bakery                  25
Thai Restaurant         25
Name: Venue Category, dtype: int64

## Analyzing frequency

In [43]:
downtown_dummies = pd.get_dummies(dt_toronto_venues[["Venue Category"]],prefix="", prefix_sep="")

downtown_dummies["Neighborhood"]=dt_toronto_venues["Neighborhood"]

col_names = [downtown_dummies.columns[-1]]+list(downtown_dummies.columns[:-1])

downtown_dummies=downtown_dummies[col_names]

In [44]:
downtown_dummies.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [110]:
downtown_dummies.shape

(1220, 181)

### Venues Frequency by Neighborhood

In [111]:
downtown_grouped=downtown_dummies.groupby("Neighborhood").mean().reset_index()

In [112]:
downtown_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Bay Street,0.000000,0.000000,0.000000,0.010000,0.00,0.010000,0.01,0.000000,0.000000,...,0.00,0.00,0.010000,0.000000,0.010000,0.010000,0.01,0.000000,0.000000,0.00
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.015152,0.000000,0.000000,0.015152,0.00,0.000000,0.00,0.000000,0.015152,...,0.00,0.00,0.015152,0.015152,0.000000,0.000000,0.00,0.000000,0.000000,0.00
2,Church and Wellesley,0.000000,0.012195,0.012195,0.024390,0.00,0.000000,0.00,0.012195,0.000000,...,0.00,0.00,0.012195,0.000000,0.000000,0.012195,0.00,0.012195,0.012195,0.00
3,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.040000,0.00,0.010000,0.00,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.00
4,Downtown Toronto,0.000000,0.000000,0.000000,0.031496,0.00,0.007874,0.00,0.000000,0.007874,...,0.00,0.00,0.007874,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
5,"First Canadian Place, Underground city",0.000000,0.000000,0.000000,0.040000,0.00,0.010000,0.00,0.000000,0.010000,...,0.00,0.01,0.010000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.00
6,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.010000,0.00,0.010000,0.00,0.000000,0.000000,...,0.00,0.00,0.010000,0.000000,0.010000,0.010000,0.01,0.000000,0.000000,0.00
7,"Harbourfront, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.100000,0.00,0.000000,0.00,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
8,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.000000,0.000000,0.01,0.020000,0.00,0.010000,0.000000,...,0.01,0.00,0.050000,0.000000,0.000000,0.050000,0.01,0.000000,0.000000,0.00
9,"Regent Park, Harbourfront",0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00


### Top 5 venues for each neighborhood

In [113]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['category','frequency']
    temp = temp.iloc[1:]
    temp['frequency'] = temp['frequency'].astype(float)
    temp = temp.round({'frequency': 2})
    print(temp.sort_values('frequency', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bay Street----
             category  frequency
0         Coffee Shop       0.11
1      Clothing Store       0.08
2      Cosmetics Shop       0.03
3               Plaza       0.03
4  Italian Restaurant       0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport----
             category  frequency
0         Coffee Shop       0.08
1  Italian Restaurant       0.08
2                 Pub       0.05
3                Café       0.05
4                 Bar       0.05


----Church and Wellesley----
              category  frequency
0          Coffee Shop       0.07
1  Japanese Restaurant       0.05
2              Gay Bar       0.05
3         Burger Joint       0.04
4           Restaurant       0.04


----Commerce Court, Victoria Hotel----
              category  frequency
0          Coffee Shop       0.12
1                Hotel       0.07
2                 Café       0.06
3           Restaurant       0.05
4  American Restaurant       0.04


----Do

### Create dataframe with top 5 venues for each neighborhood

In [121]:
def most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [127]:
num_top_venues = 5


columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
        columns.append('{}'.format(ind+1,))
    
        
downtown_venues_sorted = pd.DataFrame(columns=columns)
                       
downtown_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for i in np.arange(len(downtown_grouped)):
    downtown_venues_sorted.iloc[i, 1:] = most_common_venues(downtown_grouped.iloc[i, :], num_top_venues)

downtown_venues_sorted

,Neighborhood,1,2,3,4,5
0,Bay Street,Coffee Shop,Clothing Store,Cosmetics Shop,Plaza,Italian Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Pub,Bar,Restaurant
2,Church and Wellesley,Coffee Shop,Gay Bar,Japanese Restaurant,Burger Joint,Restaurant
3,"Commerce Court, Victoria Hotel",Coffee Shop,Hotel,Café,Restaurant,American Restaurant
4,Downtown Toronto,Coffee Shop,Café,Steakhouse,Hotel,Bar
5,"First Canadian Place, Underground city",Coffee Shop,Hotel,Café,Steakhouse,American Restaurant
6,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Bar,Hotel
7,"Harbourfront, Union Station, Toronto Islands",Theme Park,Pizza Place,Light Rail Station,Beach,Scenic Lookout
8,"Kensington Market, Chinatown, Grange Park",Café,Chinese Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bar
9,"Regent Park, Harbourfront",Coffee Shop,Breakfast Spot,Bakery,Café,Park


## Clustering

### Importing libraries

In [115]:
from sklearn.cluster import KMeans

In [114]:
k=5

downtown_clustering = downtown_grouped.drop("Neighborhood", axis=1)

In [116]:
downtown_clustering.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0.000000,0.000000,0.000000,0.010000,0.0,0.010000,0.01,0.000000,0.000000,0.000000,...,0.0,0.0,0.010000,0.000000,0.01,0.010000,0.01,0.000000,0.000000,0.0
1,0.015152,0.000000,0.000000,0.015152,0.0,0.000000,0.00,0.000000,0.015152,0.000000,...,0.0,0.0,0.015152,0.015152,0.00,0.000000,0.00,0.000000,0.000000,0.0
2,0.000000,0.012195,0.012195,0.024390,0.0,0.000000,0.00,0.012195,0.000000,0.000000,...,0.0,0.0,0.012195,0.000000,0.00,0.012195,0.00,0.012195,0.012195,0.0
3,0.000000,0.000000,0.000000,0.040000,0.0,0.010000,0.00,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.01,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.031496,0.0,0.007874,0.00,0.000000,0.007874,0.007874,...,0.0,0.0,0.007874,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0


### Run the algorithm

In [118]:
kmeans=KMeans(n_clusters=k)
kmeans.fit(downtown_clustering)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

#### Labels

In [119]:
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 2, 4, 3, 0, 1, 0, 0, 0, 0], dtype=int32)

## Final table

In [135]:
downtown_final=downtown_t.groupby("Neighborhood").mean().reset_index()

downtown_final["Cluster"]=kmeans.labels_

downtown_final=downtown_final.join(downtown_venues_sorted.set_index("Neighborhood"), on="Neighborhood")

In [136]:
downtown_final

,Neighborhood,Latitude,Longitude,Cluster,1,2,3,4,5
0,Bay Street,43.656078,-79.384918,0,Coffee Shop,Clothing Store,Cosmetics Shop,Plaza,Italian Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",43.640816,-79.399536,0,Coffee Shop,Italian Restaurant,Pub,Bar,Restaurant
2,Church and Wellesley,43.666336,-79.381195,0,Coffee Shop,Gay Bar,Japanese Restaurant,Burger Joint,Restaurant
3,"Commerce Court, Victoria Hotel",43.648403,-79.379227,0,Coffee Shop,Hotel,Café,Restaurant,American Restaurant
4,Downtown Toronto,43.653623,-79.392680,0,Coffee Shop,Café,Steakhouse,Hotel,Bar
5,"First Canadian Place, Underground city",43.647984,-79.383484,0,Coffee Shop,Hotel,Café,Steakhouse,American Restaurant
6,"Garden District, Ryerson",43.657467,-79.377708,0,Coffee Shop,Clothing Store,Café,Bar,Hotel
7,"Harbourfront, Union Station, Toronto Islands",43.617500,-79.372826,2,Theme Park,Pizza Place,Light Rail Station,Beach,Scenic Lookout
8,"Kensington Market, Chinatown, Grange Park",43.653511,-79.397224,4,Café,Chinese Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bar
9,"Regent Park, Harbourfront",43.655220,-79.361969,3,Coffee Shop,Breakfast Spot,Bakery,Café,Park


## Cluster map

### Importing libraries

In [138]:
import matplotlib.cm as cm
import matplotlib.colors as colors

### MAP

In [141]:
cluster_map_dt = folium.Map(location=[latitude, longitude], zoom_start=13,tiles='OpenStreetMap')

x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(downtown_final['Latitude'], downtown_final['Longitude'], downtown_final['Neighborhood'], downtown_final['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster: ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(cluster_map_dt)
       
cluster_map_dt

# Examine Clusters

### Cluster 1

In [157]:
downtown_final.loc[downtown_final["Cluster"]==0, downtown_final.columns[[0] + list(range(4,downtown_final.shape[1]))]]

,Neighborhood,1,2,3,4,5
0,Bay Street,Coffee Shop,Clothing Store,Cosmetics Shop,Plaza,Italian Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Pub,Bar,Restaurant
2,Church and Wellesley,Coffee Shop,Gay Bar,Japanese Restaurant,Burger Joint,Restaurant
3,"Commerce Court, Victoria Hotel",Coffee Shop,Hotel,Café,Restaurant,American Restaurant
4,Downtown Toronto,Coffee Shop,Café,Steakhouse,Hotel,Bar
5,"First Canadian Place, Underground city",Coffee Shop,Hotel,Café,Steakhouse,American Restaurant
6,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Bar,Hotel
10,"Richmond, King",Coffee Shop,Café,Restaurant,Hotel,American Restaurant
12,St. James Town,Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant
13,"St. James Town, Cabbagetown",Coffee Shop,Restaurant,Pizza Place,Indian Restaurant,Park


### Cluster 2

In [158]:
downtown_final.loc[downtown_final["Cluster"]==1, downtown_final.columns[[0] + list(range(4,downtown_final.shape[1]))]]

,Neighborhood,1,2,3,4,5
11,Rosedale,Playground,Gym / Fitness Center,Park,Candy Store,Bank


### Cluster 3

In [159]:
downtown_final.loc[downtown_final["Cluster"]==2, downtown_final.columns[[0] + list(range(4,downtown_final.shape[1]))]]

,Neighborhood,1,2,3,4,5
7,"Harbourfront, Union Station, Toronto Islands",Theme Park,Pizza Place,Light Rail Station,Beach,Scenic Lookout


### Cluster 4

In [160]:
downtown_final.loc[downtown_final["Cluster"]==3, downtown_final.columns[[0] + list(range(4,downtown_final.shape[1]))]]

,Neighborhood,1,2,3,4,5
9,"Regent Park, Harbourfront",Coffee Shop,Breakfast Spot,Bakery,Café,Park


### Cluster 5

In [162]:
downtown_final.loc[downtown_final["Cluster"]==4, downtown_final.columns[[0] + list(range(4,downtown_final.shape[1]))]]

,Neighborhood,1,2,3,4,5
8,"Kensington Market, Chinatown, Grange Park",Café,Chinese Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bar
